# Leaderboard Data like AlpacaEval 

In [1]:
import os
from datasets import load_from_disk, Audio

In [2]:
ds = load_from_disk("data-chatbot-arena-spoken-style-11labs")
print("len(ds):", len(ds))
print("ds.column_names:", ds.column_names)

len(ds): 654
ds.column_names: ['id', 'original_id', 'winner_content', 'winner_style', 'question_original', 'assistant_a', 'assistant_b', 'model_text_a', 'model_text_b', 'style']


In [3]:
def add_questions_refined(ex):
    id = ex['id']
    refined_question_path = f"./refined_questions/{id}.refined_question.txt"
    with open(refined_question_path) as f:
        refined_question = f.read().strip('"').strip()

    # None if rejected
    path = f"./refined_questions_v2/verdict/{id}.verdict.txt"
    # check if file exists
    if not os.path.exists(path):
        ex['question_refined'] = None
        # print(path, "not found")
        return ex
    
    with open(path) as f:
        x = f.read().strip()
    if x == "VALID":
        ex['question_refined'] = refined_question
    elif x == "INVALID":
        ex['question_refined'] = None
    else:
        raise Exception()
    return ex
ds = ds.map(add_questions_refined)
ds = ds.filter(lambda x: x['question_refined'] is not None)
print("len(ds):", len(ds))


len(ds): 591


In [4]:
ds[0]

{'id': 5434,
 'original_id': '15793788be324bd588e7b9707615c279',
 'winner_content': 'model_b',
 'winner_style': 'model_b',
 'question_original': 'write a blog about eLearning',
 'assistant_a': 'The world of eLearning is rapidly growing, and it is becoming increasingly important for businesses to stay competitive. As more people turn to eLearning as a way to learn new skills, companies must find ways to make their eLearning programs engaging and effective. To do this, companies must focus on creating courses that are visually appealing and easy to navigate, while also providing engaging content that keeps learners engaged and motivated. Additionally, companies must ensure that their eLearning platform is secure, allowing learners to access their courses from any device with an internet connection. By investing in these strategies, companies can create eLearning programs that are both effective and engaging, ensuring that their employees are equipped with the skills they need to succeed.

In [5]:
def add_question_refined_wav(ex):
    id = ex['id']
    path = f"./refined_questions_kokoro_wav/{id}.wav"
    ex['question_refined_wav'] = path
    return ex

def add_model_type(ex):
    ex['model_type'] = ex['winner_style']
    return ex

def add_assistant_wav(ex):
    id = ex['id']
    winner = ex['winner_style']
    if winner == 'model_a':
        path = f"./generated_11labs_audio/{id}_model_a.11labs.wav"
    elif winner == 'model_b':    
        path = f"./generated_11labs_audio/{id}_model_b.11labs.wav"
    else:
        raise Exception()
    if os.path.exists(path):
        ex['assistant_wav'] = path
    else:
        ex['assistant_wav'] = None
    return ex
    
ds = ds.map(add_question_refined_wav)
ds = ds.map(add_model_type)
ds = ds.map(add_assistant_wav)
ds = ds.filter(lambda x: x['assistant_wav'] is not None)

In [6]:
print("len(ds):", len(ds))

len(ds): 591


In [7]:
ds[0]

{'id': 5434,
 'original_id': '15793788be324bd588e7b9707615c279',
 'winner_content': 'model_b',
 'winner_style': 'model_b',
 'question_original': 'write a blog about eLearning',
 'assistant_a': 'The world of eLearning is rapidly growing, and it is becoming increasingly important for businesses to stay competitive. As more people turn to eLearning as a way to learn new skills, companies must find ways to make their eLearning programs engaging and effective. To do this, companies must focus on creating courses that are visually appealing and easy to navigate, while also providing engaging content that keeps learners engaged and motivated. Additionally, companies must ensure that their eLearning platform is secure, allowing learners to access their courses from any device with an internet connection. By investing in these strategies, companies can create eLearning programs that are both effective and engaging, ensuring that their employees are equipped with the skills they need to succeed.

In [8]:
ds_filtered = ds.select_columns(
    ["id", "original_id", "model_type", "question_refined", "question_refined_wav", "assistant_wav"]
).cast_column("question_refined_wav", Audio()).cast_column("assistant_wav", Audio())

In [ ]:
# from huggingface_hub import login
# login(token="")

In [11]:
ds_filtered.push_to_hub("potsawee/chatbot-arena-spoken-style-eval-591", max_shard_size="100MB")

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/59 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/493 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/potsawee/chatbot-arena-spoken-style-eval-591/commit/46a21c3e345aba19a1941287ba96916b3070aec4', commit_message='Upload dataset', commit_description='', oid='46a21c3e345aba19a1941287ba96916b3070aec4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/potsawee/chatbot-arena-spoken-style-eval-591', endpoint='https://huggingface.co', repo_type='dataset', repo_id='potsawee/chatbot-arena-spoken-style-eval-591'), pr_revision=None, pr_num=None)